<a href="https://colab.research.google.com/github/nsstnaka/machine_learning_handson/blob/master/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 機械学習ハンズオン（線形回帰編）

## 事前準備

ライブラリをロードします。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## 単回帰分析

【課題】賃貸アパートの家賃を予測してみましょう。徒歩10分では家賃はいくらでしょうか？

| 駅からの時間（分） | 家賃（万円）|
| ---: | ---: |
| 1 | 8.02 |
| 2 | 7.75 |
| 3 | 7.63 |
| 4 | 7.47 |
| 5 | 7.09 |
| 6 | 7.01 |
| 7 | 6.75 |
| 8 | 6.65 |
| 9 | 6.39 |
|**10** | **????**|


### データの作成

まず、$x$と$y$のデータを作ります。
この課題では、$x$は駅からの時間、$y$は家賃とします。

In [ ]:
xs = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]
ys = [8.02, 7.75, 7.63, 7.47, 7.09, 7.01, 6.75, 6.65, 6.39]

これをグラフで可視化してみましょう。どんな傾向が見えてくるでしょうか？

In [ ]:
%matplotlib inline
plt.scatter(xs, ys)
plt.xlabel("x")
plt.ylabel("y")

In [ ]:
%matplotlib inline
plt.plot([0.0, 9.0], [8.2, 6.4], color='r', ls=':')
plt.scatter(xs, ys)
plt.xlabel("x")
plt.ylabel("y")

### 学習

上で作ったデータを使って、重みとバイアスを学習していきます。

予測を行う関数を定義しておきます。

In [ ]:
def predict(x: float, w: float, b: float) -> float:
    return w * x + b

各種設定（**ハイパーパラメータ**の設定）を行います。

In [ ]:
init_weight = 0.0  # 重みの初期値
init_bias = 0.0  # バイアスの初期値
learning_rate = 0.03  # 学習率
epochs = 1000  # 学習回数（同じデータをこの回数だけ繰り返し学習させる）

重みとバイアスを初期化します。また、途中経過を記録するための空の配列を用意しておきます。

※学習をやり直すときは、ここから下のセルを再実行します。

In [ ]:
# 重みとバイアスの初期化
weight = init_weight
bias = init_bias

# 記録用の配列
loss_history = []
weight_history = [init_weight]
bias_history = [init_bias]

いよいよ学習を実行します。

In [ ]:
for _ in range(epochs):
    sum_squared_error = 0.0  # 損失の合計（各データの損失を合計し、あとで平均を求める）
    sum_grad_w = 0.0  # 重みの勾配の合計（各データの勾配を合計し、あとで平均を求める）
    sum_grad_b = 0.0  # バイアスの勾配の合計（各データの勾配を合計し、あとで平均を求める）

    # データ1個ごとに家賃を予測し、損失および勾配を計算する
    for x, y in zip(xs, ys):  # xsとysからxとyを1個ずつ取り出す
        # 予測値を計算
        pred_value = predict(x, weight, bias)

        # 損失と勾配を計算
        error = pred_value - y
        sum_squared_error += error ** 2  # "a ** b" は「aのb乗」
        sum_grad_w += x * error
        sum_grad_b += error

    # 損失（平均二乗誤差）を求める
    loss = sum_squared_error / len(xs)

    # 重みとバイアスの勾配の平均値を求める
    grad_w = sum_grad_w / len(xs)
    grad_b = sum_grad_b / len(xs)

    # 重みとバイアスを更新
    weight -= learning_rate * grad_w
    bias -= learning_rate * grad_b

    # グラフ出力用に損失と勾配の履歴を残す
    loss_history.append(loss)
    weight_history.append(weight)
    bias_history.append(bias)


学習回数と損失の関係をグラフに表してみましょう。（これを **学習曲線(Learning curve)** と呼びます）

回数を重ねるごとに損失が減っていくのがわかります。

In [ ]:
%matplotlib inline
plt.plot(range(1, epochs+1), loss_history)
plt.title("Learning curve")
plt.xlabel("epoch")
plt.ylabel("loss")

学習が進むについて重みとバイアスが調整されていく様子を可視化してみましょう。

In [ ]:
%matplotlib inline
plt.figure(figsize=(12, 18))
# 学習回数の10分の1ごとにグラフ出力
for i, epoch in enumerate(range(int(epochs/10), epochs+1, int(epochs/10))):
    ax = plt.subplot(5, 2, i+1)
    ax.set_title(f"{epoch} epochs")
    ax.scatter(xs, ys)
    ax.set_ylim(4.0, 9.0)
    ax.plot([0, 10], [bias_history[epoch], predict(10.0, weight_history[epoch], bias_history[epoch])], c='r', ls=':')


学習が終わった時点での重みとバイアスを見てみましょう。

重みが-0.2、バイアスが8.2に近い値になっていれば、学習がうまくいっていることになります。

In [ ]:
print(f"(weight, bias) = ({weight:.5f}, {bias:.5f})")

### 予測の実行

いよいよ、徒歩10分の家賃を予測してみましょう。

6.2万円に近ければ、予測がうまくいっていることになります。

In [ ]:
predict(10.0, weight, bias)

## 重回帰分析

【課題】引き続き、賃貸アパートの家賃を予測してみましょう。徒歩10分で2階以上の部屋の家賃はいくらでしょうか？

| 駅からの時間（分） | 2階以上? | 家賃（万円）|
| ---: | --- | ---: |
| 1 | Yes | 8.11 |
| 1 | No | 8.02 |
| 2 | Yes | 7.91 |
| 2 | No | 7.75 |
| 3 | Yes | 7.70 |
| 3 | No | 7.63 |
| 4 | Yes | 7.55 |
| 4 | No | 7.47 |
| 5 | Yes | 7.32 |
| 5 | No | 7.09 |
| 6 | Yes | 7.08 |
| 6 | No | 7.01 |
| 7 | Yes | 6.98 |
| 7 | No | 6.75 |
| 8 | Yes | 6.83 |
| 8 | No | 6.65 |
| 9 | Yes | 6.47 |
| 9 | No | 6.39 |
|**10** | **Yes** | **????**|

この課題では、**numpy**という数値計算ライブラリを使って実装することにしましょう。

### データの作成

numpyでデータを作ります。

$X$は、データが2種類（駅からの時間と2階以上かどうか）あるため、二次元配列にします。

なお、データは数値にしなければならないため、2階以上かどうかは、Yesなら1、Noなら0の数値に変換しておきます。

In [ ]:
xs_multi = np.array([[1.0, 1.0],
                     [1.0, 0.0], 
                     [2.0, 1.0], 
                     [2.0, 0.0],
                     [3.0, 1.0],
                     [3.0, 0.0],
                     [4.0, 1.0],
                     [4.0, 0.0],
                     [5.0, 1.0],
                     [5.0, 0.0],
                     [6.0, 1.0],
                     [6.0, 0.0],
                     [7.0, 1.0],
                     [7.0, 0.0],
                     [8.0, 1.0],
                     [8.0, 0.0],
                     [9.0, 1.0],
                     [9.0, 0.0]], dtype=np.float32)
ys_multi = np.array([8.11, 8.02, 7.91, 7.75, 7.70, 7.63, 7.55, 7.47, 7.32, 7.09,
                     7.08, 7.01, 6.98, 6.75, 6.83, 6.65, 6.47, 6.39],
                    dtype=np.float32)
ys_multi = np.expand_dims(ys_multi, axis=1)

こちらもグラフで可視化してみましょう。どんな傾向が見えてくるでしょうか？

In [ ]:
%matplotlib inline
plt.scatter(xs_multi[xs_multi[:, 1]==1.0][:, 0],
            ys_multi[xs_multi[:, 1]==1.0][:, 0],
            label="2F or upper")
plt.scatter(xs_multi[xs_multi[:, 1]==0.0][:, 0],
            ys_multi[xs_multi[:, 1]==0.0][:, 0],
            label="1F")
plt.legend()
plt.xlabel("x")
plt.ylabel("y")

### モデルの学習

データを1000回学習させ、損失をグラフ化します。

先ほどと同様に予測関数を作っておきます。
今回はすべてのデータの予測値を一度に計算してしまうことに注目してください。

In [ ]:
def predict_np(X: np.array, W: np.array, b: float) -> np.array:
    return np.dot(X, W.T) + b

ハイパーパラメータの設定を行います。

In [ ]:
init_weights = np.zeros((1, 2))  # 2個の重みをそれぞれ0に設定
init_bias = 0.0  # バイアスの初期値
learning_rate = 0.03  # 学習率
epochs = 1000  # 学習回数（同じデータをこの回数だけ繰り返し学習させる）

重みとバイアスを初期化します。また、途中の損失を記録するための空の配列を用意しておきます。

※学習をやり直すときは、ここから下のセルを再実行します。

In [ ]:
# 重みとバイアスの初期化
weights = np.copy(init_weights)
bias = init_bias

# 記録用の配列
loss_history = []

学習を実行します。

In [ ]:
for e in range(epochs):
    # 各データの予測値を一度にすべて求める
    pred_values = predict_np(xs_multi, weights, bias)

    # 損失（平均二乗誤差）を求める
    error = pred_values - ys_multi
    loss = np.mean(error ** 2)

    # 勾配を求める
    grad_ws = np.dot(error.T, xs_multi) / xs_multi.shape[0]
    grad_b = np.mean(error)

    # 重みとバイアスを更新
    weights -= learning_rate * grad_ws
    bias -= learning_rate * grad_b

    # グラフ出力用に損失と勾配の履歴を残す
    loss_history.append(loss)
    weight_history.append(weights)
    bias_history.append(bias)


学習曲線を表してみます。先ほど同様、回数を重ねるごとに損失が減っていくのがわかります。

In [ ]:
%matplotlib inline
plt.plot(range(1, epochs+1), loss_history)
plt.title("Learning curve")
plt.xlabel("epoch")
plt.ylabel("loss")

重みとバイアスを見てみます。重みが(-0.2, +0.1)、バイアスが8.2に近い値になっていれば、学習がうまくいっていることになります。

In [ ]:
print(f"(weight, bias) = ({weights}, {bias:.5f})")

### 予測の実行

最後に、徒歩10分で2階以上の物件の家賃を予測してみましょう。

6.3万円に近ければ、予測がうまくいっていることになります。

In [ ]:
predict_np([10.0, 1.0], weights, bias)

# おまけ：ライブラリを使った実装

【課題】重回帰分析の課題を、ディープラーニングライブラリの一つである**TensorFlow**を使って実装してみましょう。

なお、$X$と$Y$のデータは上記の重回帰分析で作ったものをそのまま使うことにします。

In [ ]:
import tensorflow as tf

まず**モデル**を作ります。モデルに対してデータを流し込むことによって学習を進めることになります。

2行目の `Dense` が重みとバイアスを持ち、$\hat{Y}=W^TX+b$を計算します。
4行目のcompileでモデルを組み立てます。`optimizer`でパラメータの更新方法を（ここに学習率を渡しています）、`loss`で損失（ここでは平均二乗誤差）を指定しています。

※学習をやり直すときは、ここから下のセルを再実行します。

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_dim=2)
])
model.compile(optimizer=tf.keras.optimizers.SGD(0.03), loss="mean_squared_error")

作ったモデルの中身を見てみましょう。

In [ ]:
model.summary()

モデルの学習を行います。同じデータを1000回学習させます。

損失の計算、勾配の計算、パラメータの更新は自動的に行われます。

In [ ]:
hist = model.fit(xs_multi, ys_multi, epochs=1000, verbose=0)

学習曲線を可視化します。

In [ ]:
%matplotlib inline
plt.plot(range(1, 1001), hist.history["loss"])
plt.title("Learning rate")
plt.xlabel("epoch")
plt.ylabel("loss")

重みとバイアスを見てみます。重みが(-0.2, +0.1)、バイアスが8.2に近い値になっていれば、学習がうまくいっていることになります。

In [ ]:
weights = model.layers[0].get_weights()[0][0]
biases = model.layers[0].get_weights()[1][0]
print(f"(weights, bias) = ({weights}, {biases})")

最後に予測をしてみましょう。6.3万円に近ければ予測がうまくいっていることになります。

In [ ]:
model.predict([[10.0, 1.0]])

## 研究課題

上のそれぞれの課題において、学習率(`learning_rate`)をいろいろ変えて試してみましょう。学習曲線や予測結果にどのような影響があるでしょうか？